# Notebook 3 : SQL

In [ ]:
# Décommenter la ligne suivante pour installer ibis
# %pip install ibis-framework[sqlite]

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import sqlite3

import pandas as pd
import ibis

from ibis import _

ibis.options.interactive = True

query_tables = "SELECT name FROM sqlite_master WHERE type='table'"

## STAR

Nous considérons les données des stations de vélos en libre service [STAR](https://www.star.fr/) de Rennes Métropole. Une copie de la base SQLite est disponible dans le fichier `star.db`. Nous utilisons d'abord Pandas pour répondre aux questions, puis Ibis.

1. Se connecter à la base de données et afficher la liste des tables à l'aide de la fonction `read_sql` de Pandas et de la requête `query_tables`.

In [4]:
con = sqlite3.connect("C:/Users/cepe-s3-02/Desktop/David/Bloc 2/notebooks/data/star.db")

print(pd.read_sql(query_tables, con))

        name
0  Topologie
1       Etat


2. Récupérer le contenu de la table `Etat` dans un dataframe et afficher la liste des variables disponibles. Même question pour la table `Topologie`.

In [13]:
df_etat = pd.read_sql("SELECT * FROM Etat", con)
df_etat.dtypes

print(df_etat.dtypes)

df_topologie = pd.read_sql("SELECT * FROM Topologie", con)
df_topologie.dtypes

print(df_topologie.dtypes)

id                            int64
nom                          object
latitude                    float64
longitude                   float64
etat                         object
nb_emplacements               int64
emplacements_disponibles      int64
velos_disponibles             int64
date                        float64
data                         object
dtype: object
id                     int64
nom                   object
adresse_numero        object
adresse_voie          object
commune               object
latitude             float64
longitude            float64
id_correspondance    float64
mise_en_service      float64
nb_emplacements        int64
id_proche_1            int64
id_proche_2            int64
id_proche_3            int64
terminal_cb           object
dtype: object


3. Sélectionner l'identifiant `id`, le nom `nom` et l'identifiant de la station la plus proche `id_proche_1` depuis la table `Topologie`.²

In [12]:
query_base = """
    SELECT
    id,
    nom
    FROM Topologie
    WHERE
    id_proche_1=1
"""

df_base = pd.read_sql(query_base, con)
df_base.dtypes
print(df_base)

   id     nom
0   2  Mairie


4. Faire une jointure sur la table précédente pour créer une table qui contient la liste des stations avec l'identifiant, le nom et le nom de la station la plus proche associée à l'identifiant `id_proche_1`. Les variables utilisées comme clés sont différents, penser à utiliser les arguments `left_on` et `right_on` de la méthode `merge`.

In [36]:
query_base = """
    SELECT t1.id, t1.nom, t2.nom AS nom_proche
    FROM Topologie AS t1
    LEFT JOIN Topologie AS t2 ON t1.id_proche_1 = t2.id
    WHERE t1.id_proche_1=1
"""

df_base = pd.read_sql(query_base, con)
df_base.dtypes
#print(df_base)

# Effectuer la jointure avec merge
df_base2 = pd.merge(
    df_topologie,  # DataFrame de gauche
    df_topologie,  # DataFrame de droite
    left_on="id_proche_1",  # Clé de jointure dans le DataFrame de gauche
    right_on="id",  # Clé de jointure dans le DataFrame de droite
    suffixes=("_t1", "_t2")  # Suffixes pour différencier les colonnes
)

#print (df_base2)

# Filtrer les résultats pour id_proche_1 = 1
# df_base2 = df_base2[df_base2["id_proche_1_t1"] == 1]

# Sélectionner les colonnes nécessaires
df_base2 = df_base2[["id_t1", "nom_t1", "nom_t2", "latitude_t1", "longitude_t1", "latitude_t2", "longitude_t2"]]
# df_base2.columns = ["id", "nom", "nom_proche"]  # Renommer les colonnes

# Afficher le résultat
print(df_base2)

    id_t1             nom_t1              nom_t2  latitude_t1  longitude_t1  \
0       1         République              Mairie    48.110026     -1.678037   
1       3      Champ Jacquet              Mairie    48.112764     -1.680062   
2       9      Saint-Georges              Mairie    48.112385     -1.674417   
3       2             Mairie          République    48.111624     -1.678757   
4      10   Musée Beaux-Arts                 TNB    48.109601     -1.674080   
..    ...                ...                 ...          ...           ...   
76     11            Liberté          Les Halles    48.107514     -1.678163   
77     19    Plélo Colombier          Les Halles    48.105897     -1.681374   
78     24  Place de Bretagne  Office de Tourisme    48.109621     -1.684019   
79     55         Préfecture             Cucillé    48.128453     -1.694032   
80     69    Champs Manceaux   Bréquigny Piscine    48.091114     -1.682284   

    latitude_t2  longitude_t2  
0     48.111624    

5. Ajouter à la table précédente la distance entre la station et la station la plus proche.

In [37]:
import math

def haversine(lat1, lon1, lat2, lon2):
    """
    Calcule la distance entre deux points géographiques en utilisant la formule de Haversine.
    Les coordonnées doivent être en degrés.
    
    :param lat1: Latitude du premier point
    :param lon1: Longitude du premier point
    :param lat2: Latitude du second point
    :param lon2: Longitude du second point
    :return: Distance en kilomètres
    """
    # Rayon moyen de la Terre en kilomètres
    R = 6371.0

    # Convertir les degrés en radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Différences des coordonnées
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Formule de Haversine
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Distance
    distance = R * c
    return distance

def distance(lat1, lon1, lat2, lon2):
    distance = ((lat2 - lat1)**2 + (lon2 - lon1)**2)**0.5
    return distance

# Appliquer la fonction haversine sur chaque ligne du DataFrame
df_base2["distance"] = df_base2.apply(
    lambda row: distance(row["latitude_t1"], row["longitude_t1"], row["latitude_t2"], row["longitude_t2"]),
    axis=1
)

# Afficher le DataFrame avec la colonne distance
print(df_base2)


    id_t1             nom_t1              nom_t2  latitude_t1  longitude_t1  \
0       1         République              Mairie    48.110026     -1.678037   
1       3      Champ Jacquet              Mairie    48.112764     -1.680062   
2       9      Saint-Georges              Mairie    48.112385     -1.674417   
3       2             Mairie          République    48.111624     -1.678757   
4      10   Musée Beaux-Arts                 TNB    48.109601     -1.674080   
..    ...                ...                 ...          ...           ...   
76     11            Liberté          Les Halles    48.107514     -1.678163   
77     19    Plélo Colombier          Les Halles    48.105897     -1.681374   
78     24  Place de Bretagne  Office de Tourisme    48.109621     -1.684019   
79     55         Préfecture             Cucillé    48.128453     -1.694032   
80     69    Champs Manceaux   Bréquigny Piscine    48.091114     -1.682284   

    latitude_t2  longitude_t2  distance  
0     48.

6. Créer une table avec le nom des trois stations les plus proches du point GPS *(48.1179151,-1.7028661)* classées par ordre de distance et le nombre de vélos disponibles dans ces stations.

In [44]:
df_topologie["DistanceGPS"] = df_topologie.apply(
    lambda row: distance(row["latitude"], row["longitude"], 48.1179151, -1.7028661),
    axis=1
)

#print(df_topologie[["id", "nom", "DistanceGPS"]].sort_values("DistanceGPS"))

stationsProcheGPS = df_topologie.head(3)

#print(stationsProcheGPS)

# Effectuer la jointure avec merge
df_base3 = pd.merge(
    stationsProcheGPS,  # DataFrame de gauche
    df_etat,  # DataFrame de droite
    left_on="id",  # Clé de jointure dans le DataFrame de gauche
    right_on="id",  # Clé de jointure dans le DataFrame de droite
)

df_base3 = df_base3[["id", "nom_x", "DistanceGPS", "velos_disponibles"]]

print(df_base3)

   id          nom_x  DistanceGPS  velos_disponibles
0   1     République     0.026052                  5
1   2         Mairie     0.024917                 18
2   3  Champ Jacquet     0.023379                 16


7. Reprendre les questions précédentes en utilisant le module `ibis`. Pour les jointures, utiliser la méthode `left_join`.

In [67]:
con2 = ibis.sqlite.connect("C:/Users/cepe-s3-02/Desktop/David/Bloc 2/notebooks/data/star.db")

topologie_table = con2.table("Topologie")
etat_table = con2.table("Etat")

#topologie.columns
#etat.columns

# Sélectionner l'identifiant `id`, le nom `nom` et l'identifiant de la station la plus proche `id_proche_1` depuis la table `Topologie`.

topologie_table.select(
    topologie_table.id,
    topologie_table.nom,
    topologie_table.id_proche_1
)[topologie_table.id_proche_1 == 1]

# Faire une jointure sur la table précédente pour créer une table qui contient la liste des stations avec l'identifiant, le nom et le nom de la station la plus proche associée à l'identifiant `id_proche_1`. Les variables utilisées comme clés sont différents, penser à utiliser les arguments `left_on` et `right_on` de la méthode `merge`.

"""
topologie_table.left_join(
    topologie_table,  # Table de droite
    left_on=topologie_table.id_proche_1,  # Clé de jointure dans la table de gauche
    right_on=topologie_table.right.id  # Clé de jointure dans la table de droite
)
"""
"""
    SELECT t1.id, t1.nom, t2.nom AS nom_proche
    FROM Topologie AS t1
    LEFT JOIN Topologie AS t2 ON t1.id_proche_1 = t2.id
    WHERE t1.id_proche_1=1
"""

C:\Users\cepe-s3-02\AppData\Local\Temp\ipykernel_10012\3203235617.py:11: FutureWarning: Selecting/filtering arbitrary expressions in `Table.__getitem__` is deprecated and will be removed in version 10.0. Please use `Table.select` or `Table.filter` instead.
  topologie_table.select(


'\n    SELECT t1.id, t1.nom, t2.nom AS nom_proche\n    FROM Topologie AS t1\n    LEFT JOIN Topologie AS t2 ON t1.id_proche_1 = t2.id\n    WHERE t1.id_proche_1=1\n'

8. (*Bonus*) Écrire des requêtes SQL pour obtenir les résultats demandés dans les questions 3 à 6. La fonction `to_sql` pourra être utilisée pour de l'aide.

## Musique

Le dépôt GitHub [lerocha/chinook-database](https://github.com/lerocha/chinook-database) met à disposition des bases de données de bibliothèques musicales. Une copie de la base SQLite est disponible dans le fichier `chinook.db`.

1. Utiliser le module `ibis` pour vous connecter à la base de données et explorer les tables formant le jeu de données pour le découvrir. En particulier, remarquer comment les tables `Playlist`, `PlaylistTrack` et `Track` sont liées entre elles.

In [68]:
con3 = ibis.sqlite.connect("C:/Users/cepe-s3-02/Desktop/David/Bloc 2/notebooks/data/chinook.db")

2. Quelles sont les playlists qui contiennent le plus de pistes ?

In [ ]:
tables_con3 = con3.list_tables()

print(tables_con3)

playlist = con3.table("PlaylistTrack")

print(playlist.columns)

playlist.group_by("PlaylistId").aggregate(
    count=_.count()
).order_by(ibis.desc("count"))


['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']
('PlaylistId', 'TrackId')


┏━━━━━━━━━━━━┳━━━━━━━┓
┃ PlaylistId ┃ count ┃
┡━━━━━━━━━━━━╇━━━━━━━┩
│ !int64     │ int64 │
├────────────┼───────┤
│          1 │  3290 │
│          8 │  3290 │
│          5 │  1477 │
│          3 │   213 │
│         10 │   213 │
│         12 │    75 │
│         11 │    39 │
│         17 │    26 │
│         13 │    25 │
│         14 │    25 │
│          … │     … │
└────────────┴───────┘

3. Construire une table contenant les informations suivantes sur la playlist `Classical` : le titre de chaque piste ainsi que le titre de l'album dont cette piste est tirée.

4. (*Bonus*) Écrire une requête SQL donnant le résultat de la question précédente. La fonction `to_sql` pourra être utilisée pour de l'aide.